In [1]:
import pandas as pd
import numpy as np
import janitor as jn
from janitor.utils import _generic_less_than_inequality, _generic_greater_than_inequality
from pandas.api.types import is_numeric_dtype, is_datetime64_dtype, is_string_dtype
from io import StringIO

In [2]:
df1 = pd.DataFrame(
    [
        {"x": "b", "y": 1, "v": 1},
        {"x": "b", "y": 3, "v": 2},
        {"x": "b", "y": 6, "v": 3},
        {"x": "a", "y": 1, "v": 4},
        {"x": "a", "y": 3, "v": 5},
        {"x": "a", "y": 6, "v": 6},
        {"x": "c", "y": 1, "v": 7},
        {"x": "c", "y": 3, "v": 8},
        {"x": "c", "y": 6, "v": 9},
        {"x": "c", "y": np.nan, "v": 9},
    ]
)

df2 = pd.DataFrame(
    [{"x": "c", "v": 8, "foo": 4}, {"x": "b", "v": 7, "foo": 2}, {"x": "b", "v": 7, "foo": None}]
)

# result = jn.cond_merge(df1, df2, ('y', 'foo', 'le'))



In [3]:
# df1.cond_merge(df2, ('y','foo','lt'), ('v','v', 'le'))

In [4]:
%load_ext line_profiler
%load_ext memory_profiler

In [5]:
df1 = pd.concat([df1]*1_000, ignore_index=True)
df2 = pd.concat([df2]*200, ignore_index = True)

In [29]:
df1.lt_join(df2, 'y','foo')# .query("v_x >= v_y")

KeyError: 'y'

In [7]:
# %timeit _conditional_merge_compute(df1, df2, [Inequality_Condition('y', 'foo', 'lt')], how='inner', suffixes=(None, "_y"))

In [8]:
_generic_less_than_inequality(df1, df2, 'y', 'foo', strict = True)

,x,v,foo,y
298,b,7,2.0,1.0
310,b,7,2.0,1.0
313,b,7,2.0,1.0
316,b,7,2.0,1.0
319,b,7,2.0,1.0
...,...,...,...,...
360,c,8,4.0,3.0
363,c,8,4.0,3.0
117,c,8,4.0,3.0
369,c,8,4.0,3.0


In [9]:
%lprun -f  _generic_less_than_inequality   _generic_less_than_inequality(        df=df1, right=df2, left_on="v", right_on="v"                                             , strict=True)

Timer unit: 1e-06 s

Total time: 0.05617 s
File: /workspaces/pyjanitor/janitor/utils.py
Function: _generic_less_than_inequality at line 2449

Line #      Hits         Time  Per Hit   % Time  Line Contents
  2449                                           def _generic_less_than_inequality(
  2450                                               df: pd.DataFrame,
  2451                                               right: pd.DataFrame,
  2452                                               left_on: str,
  2453                                               right_on: str,
  2454                                               strict: bool = False,
  2455                                           ):
  2456                                               """
  2457                                               Generic implementation for less than joins.
  2458                                               This covers less than or equal joins as well.
  2459                                             

In [10]:
# %lprun -f janitor.le_join janitor.le_join(df1, df2, 'y', 'foo')

In [11]:
A = pd.DataFrame(dict(
        A_id=range(10),
        A_value=range(5, 105, 10)
    ))
B = pd.DataFrame(dict(
        B_id=range(5),
        B_low=[0, 30, 30, 46, 84],
        B_high=[10, 40, 50, 54, 84]
    ))

In [12]:
A

,A_id,A_value
0,0,5
1,1,15
2,2,25
3,3,35
4,4,45
5,5,55
6,6,65
7,7,75
8,8,85
9,9,95


In [13]:
B

,B_id,B_low,B_high
0,0,0,10
1,1,30,40
2,2,30,50
3,3,46,54
4,4,84,84


In [14]:
outcome = A.le_join(B, left_on='A_value', right_on='B_high').query('A_value >= B_low')

outcome

,A_id,A_value,B_id,B_low,B_high
0,0,5,0,0,10
13,3,35,1,30,40
14,3,35,2,30,50
17,4,45,2,30,50


In [15]:
A.merge(outcome, how = 'left')

,A_id,A_value,B_id,B_low,B_high
0,0,5,0.0,0.0,10.0
1,1,15,NaN,NaN,NaN
2,2,25,NaN,NaN,NaN
3,3,35,1.0,30.0,40.0
4,3,35,2.0,30.0,50.0
5,4,45,2.0,30.0,50.0
6,5,55,NaN,NaN,NaN
7,6,65,NaN,NaN,NaN
8,7,75,NaN,NaN,NaN
9,8,85,NaN,NaN,NaN


In [16]:
df_a = pd.DataFrame([{"a": 1, "b": 4}, {"a": 2, "b": 5}, {"a": 3, "b": 6}])
df_b = pd.DataFrame([{"c": 2, "d": 7}, {"c": 3, "d": 8}])

In [17]:
df_a

,a,b
0,1,4
1,2,5
2,3,6


In [18]:
df_b

,c,d
0,2,7
1,3,8


In [19]:
df_a.merge(df_a.lt_join(df_b, "a", "c"), how = 'left')

,a,b,c,d
0,1,4,2.0,7.0
1,1,4,3.0,8.0
2,2,5,3.0,8.0
3,3,6,NaN,NaN


In [20]:
df1 = pd.DataFrame({'id': [1,1,1,2,2,3], 'value_1': [2,5,7,1,3,4]})
df2 = pd.DataFrame({'id': [1,1,1,1,2,2,2,3], 'value_2A': [0,3,7,12,0,2,3,1], 'value_2B': [1,5,9,15,1,4,6,3]})

In [21]:
df1

,id,value_1
0,1,2
1,1,5
2,1,7
3,2,1
4,2,3
5,3,4


In [22]:
df2

,id,value_2A,value_2B
0,1,0,1
1,1,3,5
2,1,7,9
3,1,12,15
4,2,0,1
5,2,2,4
6,2,3,6
7,3,1,3


In [23]:
df1.merge(df1.merge(df2).query('value_2A<=value_1<=value_2B'), how = 'left')

,id,value_1,value_2A,value_2B
0,1,2,NaN,NaN
1,1,5,3.0,5.0
2,1,7,7.0,9.0
3,2,1,0.0,1.0
4,2,3,2.0,4.0
5,2,3,3.0,6.0
6,3,4,NaN,NaN


In [24]:
data = """    company       date  measure
0         0  2010-01-01       10
1         0  2010-01-15       10
2         0  2010-02-01       10
3         0  2010-02-15       10
4         0  2010-03-01       10
5         0  2010-03-15       10
6         0  2010-04-01       10
7         1  2010-03-01        5
8         1  2010-03-15        5
9         1  2010-04-01        5
10        1  2010-04-15        5
11        1  2010-05-01        5
12        1  2010-05-15        5"""

left = pd.read_csv(StringIO(data), sep="\s{2,}", engine='python')

left['date'] = pd.to_datetime(left['date'])
left

,company,date,measure
0,0,2010-01-01,10
1,0,2010-01-15,10
2,0,2010-02-01,10
3,0,2010-02-15,10
4,0,2010-03-01,10
5,0,2010-03-15,10
6,0,2010-04-01,10
7,1,2010-03-01,5
8,1,2010-03-15,5
9,1,2010-04-01,5


In [25]:
data = """   company   end_date
0        0  2010-02-01
1        0  2010-03-15
2        1  2010-04-01
3        1  2010-05-15
"""

right = pd.read_csv(StringIO(data), sep="\s{2,}", engine='python')

right['end_date'] = pd.to_datetime(right['end_date'])
right

,company,end_date
0,0,2010-02-01
1,0,2010-03-15
2,1,2010-04-01
3,1,2010-05-15


In [26]:
left.dtypes

company             int64
date       datetime64[ns]
measure             int64
dtype: object

In [27]:
right.dtypes

company              int64
end_date    datetime64[ns]
dtype: object

In [28]:
is_numeric_dtype(left.SALARY, right.grad)

AttributeError: 'DataFrame' object has no attribute 'SALARY'

In [ ]:
left.le_join(right, 'date', 'end_date', suffixes = (None, '_y')).loc[lambda df: (df['date'] >= (df['end_date']-pd.Timedelta('30D'))) & (df.company == df.company_y), ['company', 'date', 'end_date', 'measure']].groupby(['company', 'end_date']).measure.sum()

In [ ]:
df = pd.DataFrame({'Origin': {1: 'A', 6: 'A', 11: 'A', 16: 'A', 21: 'B', 26: 'B', 31: 'C', 36: 'C'},
 'Destination': {1: 'B', 6: 'B', 11: 'C', 16: 'C', 21: 'Z', 26: 'Z', 31: 'Z', 36: 'Z'}, 
 'Dept_Time': {1: pd.Timestamp('2019-03-30 17:31:00'), 6: pd.Timestamp('2019-05-16 17:32:00'), 11: pd.Timestamp('2019-04-01 08:30:00'), 16: pd.Timestamp('2019-06-09 08:20:00'), 21: pd.Timestamp('2019-07-26 08:31:00'), 26: pd.Timestamp('2019-03-31 06:16:00'), 31: pd.Timestamp('2019-07-03 23:52:00'), 36: pd.Timestamp('2019-03-27 17:31:00')}, 'Arrv_Time': {1: pd.Timestamp('2019-03-30 23:23:00'), 6: pd.Timestamp('2019-05-16 23:22:00'), 11: pd.Timestamp('2019-04-01 14:22:00'), 16: pd.Timestamp('2019-06-09 14:18:00'), 21: pd.Timestamp('2019-07-26 14:23:00'), 26: pd.Timestamp('2019-06-18 05:00:00'), 31: pd.Timestamp('2019-07-04 05:36:00'), 36: pd.Timestamp('2019-03-27 23:23:00')}})
df

In [ ]:
df.columns

In [ ]:
janitor.lt_join?

In [ ]:
df.lt_join(df, 'Dept_Time', 'Arrv_Time').filter(['Dept_Time_x', 'Arrv_Time_y'])

In [ ]:
df1 = pd.DataFrame({'col_a': [1,2,3], 'col_b': ["A", "B", "C"]})
df1

In [ ]:
df2 = pd.DataFrame({'col_a': [0, 2, 3], 'col_c': ["Z", "X", "Y"]})
df2

In [ ]:
df1.gt_join(df2, 'col_a', 'col_a', suffixes=(None, '_y'))

In [ ]:
TableA = pd.DataFrame(np.random.rand(4, 3),
                      pd.Index(list('abcd'), name='Key'),
                      ['A', 'B', 'C']).reset_index()
TableB = pd.DataFrame(np.random.rand(4, 3),
                      pd.Index(list('aecf'), name='Key'),
                      ['A', 'B', 'C']).reset_index()

In [ ]:
TableA

In [ ]:
TableB

In [ ]:
TableA.merge(TableB, on='Key', how='outer', indicator=True).query("_merge ==['left_only', 'right_only']")

In [ ]:
A = dict(x=[1,2,2,2,4,4,4], y=[2,3,4,6,5,5,10], z=[2,1,7,4,5,6,3])
B = dict(x=[2,4],y=[3,8])
A = pd.DataFrame(A)
B = pd.DataFrame(B)

In [ ]:
A

In [ ]:
B

In [ ]:
A=pd.concat([A]*10_000, ignore_index=True)
B= pd.concat([B]*10, ignore_index=True)

In [ ]:
%%timeit
index_to_na = A.le_join(B, "x", "x", suffixes=(None, '_y')).loc[lambda df: df.y > df.y_y, ["x", "y", "z"]]
A.loc[lambda df: df.z.isin(index_to_na.z), 'z'] = pd.NA
A

In [ ]:
A

In [ ]:
left = pd.DataFrame([[1, 2, 3],
                     [4, 5, 6],
                    [7, 8, 9]],columns=['a', 'b', 'c'])


right = pd.DataFrame([[1, 2.5, -3],
                      [4, 7, -6],
                   [6, 8.5, -9]],
                    columns=['a', 'd', 'e'])

left

In [ ]:
right

In [ ]:
left.lt_join(right, 'b', 'd').query('c>d')

In [ ]:
pd.DataFrame([[1, 2, 3, 2.5, -3]],
                     columns=['a', 'b', 'c', 'd', 'e'])

In [ ]:
data = """CHR  SNP  POS
1  rs2073813  753541
1  rs3131969  754182
2  rs3131968  754192
2  rs3131967  754334
3  rs3115859  754503
3  rs3131966  900000

"""

left = pd.read_csv(StringIO(data), sep="\s{2,}", engine='python')

left

In [ ]:
data = """CHR  start  end 
1  700500  833300
2  1000  20000
2  59998  60000 
3  700000  800000

"""

right = pd.read_csv(StringIO(data), sep="\s{2,}", engine='python')

right

In [ ]:
condition1 = lambda df: (df.POS.ge(df.start)) & (df.CHR.eq(df.CHR_y))
left.le_join(right, "POS", "end", suffixes=(None, "_y")).loc[condition1, ["CHR", "SNP", "POS"]]

In [ ]:
df1 = pd.DataFrame(
    {
        'TranDate': ['2018/01/05', '2018/02/07', '2018/03/24']
        ,'Loc': [4000, 5000, 4200]
    }
)

df2 = pd.DataFrame(
    {
        'StartDate': ['2018/01/01', '2018/02/01', '2018/03/01']
        ,'EndDate': ['2018/01/31','2018/02/28', '2018/03/30']
        ,'PP': ['01', '02', '03']
    }
)

In [ ]:
df1

In [ ]:
df2

In [ ]:
dfR = pd.DataFrame(
    {
        'TranDate': ['2018/01/05', '2018/02/07', '2018/03/24']
        ,'Loc': [4000, 5000, 4200]
        ,'PP': ['01', '02', '03']
    }
)

dfR

In [ ]:

df1['TranDate'] = pd.to_datetime(df1['TranDate'])
df1.dtypes

In [ ]:
df2.StartDate = pd.to_datetime(df2.StartDate)
df2.EndDate = pd.to_datetime(df2.EndDate)

In [ ]:
df2.dtypes

In [ ]:
df1.le_join(df2, 'TranDate', 'EndDate').loc[lambda df: df.TranDate.ge(df.StartDate), ["TranDate", "Loc", "PP"]]

In [ ]:
data = """rank    begin    end     labels
first   30953   31131    label1
first   31293   31435    label2
first   31436   31733    label4
first   31734   31754    label1
first   32841   33037    label3
second  33048   33456    label4

"""

left = pd.read_csv(StringIO(data), sep="\s{2,}", engine='python')

left

In [ ]:
data = """rank    start 
first   31333     
first   31434     
first   33039    
first   33123     
first   33125     
"""

right = pd.read_csv(StringIO(data), sep="\s{2,}", engine='python')

right

In [ ]:
left

In [ ]:
right.filter(['rank', 'start', 'labels']).merge(right.le_join(left, "start", "end", suffixes=(None, "_y")).loc[lambda df: df.start.gt(df.begin), ['rank', 'start', 'labels']], how='left')

In [ ]:
%lprun -f _conditional_merge_compute _conditional_merge_compute(df1, df2, [Inequality_Condition('y', 'foo', 'lt'), Inequality_Condition('v', 'v', 'gt')], "inner", ("_x", "_y"))